In [1]:
from sqlalchemy import extract, desc
import datetime
import time
import MetaTrader5 as mt5
import pandas as pd

import options_estrategies as opt_est
from options_estrategies import Option_Due, InformationType
import models as model

In [2]:
if not mt5.initialize():
    print("initialize() failed")
    mt5.shutdown()

estrategies = opt_est.options_estrategies(model, model.PETR4, model.PETR4_OPTIONS)

In [3]:
def thl_convert_dict_from_update_ticks_to_dataframe(tick_data):
    df_tick_data = pd.DataFrame.from_dict(tick_data)
    df_tick_data = df_tick_data.drop(columns=['_sa_instance_state','timestamp_option','days_to_due_date'])
    df_tick_data['stock_price'] = df_tick_data.stock_price.astype(float)
    df_tick_data = df_tick_data[['option_name','deal_type_zone','stock_price','strike','bid','ask']]
    
    return df_tick_data


In [4]:
def thl_convert_dict_from_update_ticks_to_dataframe(data_thl):
    df_thl_compare = data_thl.copy()
    df_thl_compare["thl_cost"] = (df_thl_compare['bid_x'] - df_thl_compare['ask_y'])*-1
    df_thl_compare["thl_percent_of_strike"] = (df_thl_compare["thl_cost"] / df_thl_compare.index)*100
    df_thl_compare = df_thl_compare[['option_name_x','option_name_y','thl_percent_of_strike','thl_cost','bid_x','ask_y','deal_type_zone_y']]
    df_thl_compare = df_thl_compare[df_thl_compare['ask_y'] > 0]
    return df_thl_compare

In [8]:
returned_thl_operations = estrategies.thl_operation(mode=InformationType.Real_Time)

comparison_this_month = returned_thl_operations['thl']
comparison_next_month = returned_thl_operations['thl_next_month']
comparison_calendar = returned_thl_operations['thl_calendar']

good_operation_THL = comparison_this_month[comparison_this_month['thl_percent_of_strike'] < 1.5]
good_operation_THL_next_month = comparison_next_month[comparison_next_month['thl_percent_of_strike'] < 1.5]
good_operation_calendar = comparison_calendar[comparison_calendar['thl_percent_of_strike'] < 1.5]


good_operation_THL

,option_name_x,option_name_y,thl_percent_of_strike,thl_cost,bid_x,ask_y,deal_type_zone_y
strike,,,,,,,
28.96,PETRH243,PETRI289,1.484807,0.43,0.48,0.91,OTM


In [9]:
good_operation_THL_next_month


,option_name_x,option_name_y,thl_percent_of_strike,thl_cost,bid_x,ask_y,deal_type_zone_y
strike,,,,,,,


In [10]:
good_operation_calendar  

,option_name_x,option_name_y,thl_percent_of_strike,thl_cost,bid_x,ask_y,deal_type_zone_y
strike,,,,,,,


In [ ]:
df_thl = pd.merge(df_this_month, df_next_month, left_index=True, right_index=True)
df_thl_next_month = pd.merge(df_next_month, df_after_next_month, left_index=True, right_index=True)
df_thl_calendar = pd.merge(df_this_month, df_after_next_month, left_index=True, right_index=True)

In [ ]:
def thl_convert_dict_from_update_ticks_to_dataframe(data_thl):
    df_thl_compare = data_thl.copy()
    df_thl_compare["thl_cost"] = (df_thl_compare['bid_x'] - df_thl_compare['ask_y'])*-1
    df_thl_compare["thl_percent_of_strike"] = (df_thl_compare["thl_cost"] / df_thl_compare.index)*100
    df_thl_compare = df_thl_compare[['option_name_x','option_name_y','thl_percent_of_strike','thl_cost','bid_x','ask_y','deal_type_zone_y']]
    df_thl_compare = df_thl_compare[df_thl_compare['ask_y'] > 0]
    return df_thl_compare

In [ ]:
comparison_this_month = compare_to_dataframes_of_thl(df_thl)
comparison_next_month = compare_to_dataframes_of_thl(df_thl_next_month)
comparison_calendar = compare_to_dataframes_of_thl(df_thl_calendar)

good_operation_THL = comparison_this_month[comparison_this_month['thl_percent_of_strike'] < 1.5]
good_operation_THL_next_month = comparison_next_month[comparison_next_month['thl_percent_of_strike'] < 1.5]
good_operation_calendar = comparison_calendar[comparison_calendar['thl_percent_of_strike'] < 1.5]

if len(good_operation_THL.index) > 0:
    print(good_operation_THL)
    
if len(good_operation_THL_next_month.index) > 0:
    print(good_operation_THL_next_month)

if len(good_operation_calendar.index) > 0:
    print(good_operation_calendar) 

In [ ]:
return_of_funcion = dict(thl=comparison_this_month, thl_next_month=comparison_next_month, thl_calendar=comparison_calendar)

In [ ]:
good_operation_THL

In [ ]:
return_of_funcion['thl_next_month']